In [1]:
%pip install prettytable
%pip install pulp
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus

# Initialize problem
prob = LpProblem("AirPollutionControl", LpMinimize)

# ========== Parameters ==========
sources = [1, 2, 3, 4]
controls = [1, 2, 3, 4, 5, 6, 7, 8, 9]

Ti = {1:2, 2:3, 3:2, 4:2}

C0 = {1:1.05, 2:0.9, 3:1.1, 4:0.95, 5:0.8, 6:1.05, 7:0.95, 8:0.95, 9:0.95}
Cij = {
    (1,1): -1.0, (1,2): -1.0, (1,3): -0.95, (1,4): -1.2,
    (2,5): -1.0, (2,6): -0.85, 
    (3,7): 0.1, (3,8): 0.2,
    (4,9): 0.2
}

Tj_min = {1:2, 2:2, 3:4, 4:4, 5:3, 6:2, 7:4, 8:1, 9:1}
Tj_max = {1:9, 2:8, 3:14, 4:8, 5:10, 6:9, 7:12, 8:10, 9:10}

Cj_max = {1:0.65, 2:0.7, 3:0.5, 4:0.5, 5:0.6, 6:0.65, 7:0.65, 8:0.7, 9:0.65}
Cj_min = {1:0.4, 2:0.3, 3:0.2, 4:0.15, 5:0.2, 6:0.4, 7:0.3, 8:0.4, 9:0.4}
a = {j: -(Cj_max[j] - Cj_min[j])/(Tj_max[j] - Tj_min[j]) for j in controls}
b = {j: (Tj_max[j]*Cj_max[j] - Tj_min[j]*Cj_min[j])/(Tj_max[j] - Tj_min[j]) for j in controls}

B = 30
K = 600
T_horizon = 18
valid_pairs = list(Cij.keys())

# ========== Variables ==========
x = LpVariable.dicts("x", valid_pairs, cat=LpBinary)
Tj = LpVariable.dicts("Tj", controls, lowBound=0)
Tij = LpVariable.dicts("Tij", valid_pairs, lowBound=0)
z = LpVariable.dicts("z", controls, cat=LpBinary)
Tjz = LpVariable.dicts("Tjz", controls, lowBound=0)  # linearization helper

# ========== Constraints ==========

# z[j] reflects whether any source uses control j
for j in controls:
    prob += z[j] == lpSum(x[i, j] for i in sources if (i, j) in valid_pairs)

# source and control constraints
for i in sources:
    prob += lpSum(x[i, j] for j in controls if (i, j) in valid_pairs) <= 1
for j in controls:
    prob += lpSum(x[i, j] for i in sources if (i, j) in valid_pairs) <= 1

# time availability and activation
for (i, j) in valid_pairs:
    M = Tj_max[j]
    prob += Tj_min[j] - Ti[i] <= M * (1 - x[i, j])
    prob += Tj[j] >= Ti[i] * x[i, j]

# Tj bounds only when selected
for j in controls:
    prob += Tj[j] >= Tj_min[j] * z[j]
    prob += Tj[j] <= Tj_max[j]

# Tjz[j] = Tj[j] * z[j] linearization
for j in controls:
    Mj = Tj_max[j]
    prob += Tjz[j] <= Mj * z[j]
    prob += Tjz[j] <= Tj[j]
    prob += Tjz[j] >= Tj[j] - Mj * (1 - z[j])
    prob += Tjz[j] >= 0

# linearized Tij constraints
for (i, j) in valid_pairs:
    L1 = -T_horizon + Tj_min[j]
    U1 = -T_horizon + Tj_max[j]
    L2 = 0
    U2 = T_horizon - Tj_min[j]
    
    prob += Tij[i,j] - (T_horizon - Tj[j]) >= L1 * (1 - x[i,j])
    prob += Tij[i,j] - (T_horizon - Tj[j]) <= U1 * (1 - x[i,j])
    prob += Tij[i,j] >= L2 * x[i,j]
    prob += Tij[i,j] <= U2 * x[i,j]

# Budget constraint
prob += lpSum(
    a[j] * Tjz[j] + (b[j] + C0[j]) * x[i, j]
    for (i, j) in valid_pairs
) <= B

# Pollution reduction constraint
R = {
    (1,1): 3.00*0.95, (1,2): 3.00*0.92, 
    (1,3): 3.00*0.93, (1,4): 3.00*0.99,
    (2,5): 150.0*0.99, (2,6): 150.0*0.95,
    (3,7): 3.75*0.98, (3,8): 3.75*0.97,
    (4,9): 1.20*0.99
}
prob += lpSum(R[i, j] * Tij[i, j] for (i, j) in valid_pairs) >= K

# ========== Objective Function ==========
prob += lpSum(
    a[j]*Tjz[j] + (b[j] + C0[j])*x[i,j] + Cij[i,j]*Tij[i,j]
    for (i, j) in valid_pairs
)

# ========== Solve ==========
solver = pulp.PULP_CBC_CMD(
    timeLimit=300,
    gapRel=0.01,
    presolve=True,
    msg=True
)
prob.solve(solver)

# ========== Output ==========
print(f"Status: {LpStatus[prob.status]}")
print(f"Total Cost: ${prob.objective.value():.2f} million")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/madhavbpanicker/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/b1714d8830534143923305c5585ecdbb-pulp.mps -sec 300 -presolve on -ratio 0.01 -timeMode elapsed -branch -printingOptions all -solution /tmp/b1714d8830534143923305c5585ecdbb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 137 COLUMNS
At line 471 RHS
At line 604 BOUNDS
At line 623 ENDATA
Problem MODEL has 132 rows, 45 columns and 270 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 300
ratioGap was changed from 0 to 0.01
Option for timeMode changed from cpu to elapsed
Continuous objective value is -29 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 45 rows, 24 columns (6 integer (6 of which binary

In [ ]:
from prettytable import PrettyTable

# Table 1: Selected control actions
action_table = PrettyTable()
action_table.field_names = ["Source", "Control", "Tj (start time)", "Tij (duration)", "Reduction"]

for (i, j) in valid_pairs:
    if x[i, j].value() == 1:
        action_table.add_row([
            i, j,
            f"{Tj[j].value():.2f}",
            f"{Tij[i, j].value():.2f}",
            f"{R[i,j] * Tij[i,j].value():.2f}"
        ])

# Table 2: Control availability and linearized variable Tjz
timing_table = PrettyTable()
timing_table.field_names = ["Control", "Tj (availability)", "Tjz (used in cost)"]

for j in controls:
    if z[j].value() == 1:
        timing_table.add_row([
            j,
            f"{Tj[j].value():.2f}",
            f"{Tjz[j].value():.2f}"
        ])

# Table 3: Summary
summary_table = PrettyTable()
summary_table.field_names = ["Objective Cost", "Pollution Target", "Achieved Reduction"]
summary_table.add_row([
    f"${prob.objective.value():.2f}M",
    f"{K}",
    f"{sum(R[i, j] * Tij[i, j].value() for (i, j) in valid_pairs):.2f}"
])

# Print tables
print("\n Selected Control Assignments:")
print(action_table)

print("\n Control Timings and Linearization:")
print(timing_table)

print("\n Summary:")
print(summary_table)



✅ Selected Control Assignments:
+--------+---------+-----------------+----------------+-----------+
| Source | Control | Tj (start time) | Tij (duration) | Reduction |
+--------+---------+-----------------+----------------+-----------+
|   1    |    2    |       2.00      |     16.00      |   44.16   |
|   2    |    5    |       3.00      |     15.00      |  2227.50  |
+--------+---------+-----------------+----------------+-----------+

📈 Control Timings and Linearization:
+---------+-------------------+--------------------+
| Control | Tj (availability) | Tjz (used in cost) |
+---------+-------------------+--------------------+
|    2    |        2.00       |        2.00        |
|    5    |        3.00       |        3.00        |
+---------+-------------------+--------------------+

📊 Summary:
+----------------+------------------+--------------------+
| Objective Cost | Pollution Target | Achieved Reduction |
+----------------+------------------+--------------------+
|    $-28.00M 